In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")

In [ ]:
import requests
import re
import pandas as pd
from lxml import html

In [ ]:
def grab_player_key(player_url):
    "Returns player key from the bball reference player url"
    return re.search('(?<=/)[^/]+(?=.html)', player_url).group(0)

In [ ]:
import pandas as pd

def create_per_game_dataframe(player_url):
    """Create a dataframe with the Per Game stats of a player"""
    
    # Read html, find appropriate table
    page = requests.get(player_url)
    tree = html.fromstring(page.content)
    per_game_table_rows = tree.xpath('//*[@id="per_game"]/*/tr')
    
    # each element of per_game_cols will contain the name of the column, and the list of values in said column
    per_game_cols = []
    
    age_column_index = -1
    season_index = -1

    # Get Column Headers
    column_index = 0
    num_cols = 0
    for header in per_game_table_rows[0]:
        name = header.text_content()
        if name.upper() == "AGE":
            age_column_index = column_index
        elif name.upper() == "SEASON":
            season_index = column_index

        per_game_cols.append( (name, []) )
        column_index += 1
        num_cols += 1

    # add another column for player key
    per_game_cols.append( ("PlayerKey", []))

    if age_column_index == -1:
        raise RuntimeError("Age column index not found")
    if season_index == -1:
        raise RuntimeError("Season column index not found")
    print("Age Index: ", age_column_index)
    print("Season Index: ", season_index)

    player_key = grab_player_key(player_url)
    for row_index in range(1, len(per_game_table_rows)):
        row_element = per_game_table_rows[row_index]
        if len(row_element) != num_cols:
            continue

        # check if the column is an aggregate Career row, in which case we skip
        if row_element[age_column_index].text_content() == "":
            continue

        #Iterate through each element of the row
        column_index = 0
        for element in row_element:
            data = element.text_content()

            # format the season number to the last year
            if column_index == season_index:
                data = int(data[0:4]) + 1
            else:
                #Convert any numerical value to integers
                try:
                    data=float(data)
                except:
                    pass

            per_game_cols[column_index][1].append(data)
            column_index += 1 
        
        # add the player key to each column
        per_game_cols[column_index][1].append(player_key)

    Dict={title:column for (title,column) in per_game_cols}
    df=pd.DataFrame(Dict)
    
    #return per_game_cols
    return df


In [ ]:
url = "https://www.basketball-reference.com/players/a/antetgi01.html"
create_per_game_dataframe(url)

Age Index:  1
Season Index:  0


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PlayerKey
0,2014,19.0,MIL,NBA,SF,77.0,23.0,24.6,2.2,5.4,0.414,0.5,1.5,0.347,1.7,3.9,0.440,0.463,1.8,2.6,0.683,1.0,3.4,4.4,1.9,0.8,0.8,1.6,2.2,6.8,antetgi01
1,2015,20.0,MIL,NBA,SG,81.0,71.0,31.4,4.7,9.6,0.491,0.1,0.5,0.159,4.6,9.1,0.511,0.496,3.2,4.3,0.741,1.2,5.5,6.7,2.6,0.9,1.0,2.1,3.1,12.7,antetgi01
2,2016,21.0,MIL,NBA,PG,80.0,79.0,35.3,6.4,12.7,0.506,0.4,1.4,0.257,6.1,11.3,0.537,0.520,3.7,5.1,0.724,1.4,6.2,7.7,4.3,1.2,1.4,2.6,3.2,16.9,antetgi01
3,2017,22.0,MIL,NBA,SF,80.0,80.0,35.6,8.2,15.7,0.521,0.6,2.3,0.272,7.6,13.5,0.563,0.541,5.9,7.7,0.770,1.8,7.0,8.8,5.4,1.6,1.9,2.9,3.1,22.9,antetgi01
4,2018,23.0,MIL,NBA,PF,75.0,75.0,36.7,9.9,18.7,0.529,0.6,1.9,0.307,9.3,16.8,0.554,0.545,6.5,8.5,0.760,2.1,8.0,10.0,4.8,1.5,1.4,3.0,3.1,26.9,antetgi01
5,2019,24.0,MIL,NBA,PF,72.0,72.0,32.8,10.0,17.3,0.578,0.7,2.8,0.256,9.3,14.5,0.641,0.599,6.9,9.5,0.729,2.2,10.3,12.5,5.9,1.3,1.5,3.7,3.2,27.7,antetgi01
6,2020,25.0,MIL,NBA,PF,63.0,63.0,30.4,10.9,19.7,0.553,1.4,4.7,0.304,9.5,15.0,0.631,0.589,6.3,10.0,0.633,2.2,11.4,13.6,5.6,1.0,1.0,3.7,3.1,29.5,antetgi01


In [ ]:
player_urls = sc.textFile("player_urls.txt")
player_urls = player_urls.map(lambda x: x.strip())
player_urls.take(1)

['https://www.basketball-reference.com/players/a/abdelal01.html']

In [ ]:
def save_per_game_table(player_url):
  df = create_per_game_dataframe(player_url=player_url)
  player_key = grab_player_key(player_url=player_url)
  df.to_csv("per_game_data/" + player_key + "_per_game.csv", index=False)
  
  return df


In [ ]:
import time
tic = time.time()
b = player_urls.map(lambda x: (x, save_per_game_table(x).shape))
b.cache()
b.take(1)
toc = time.time()
print(toc-tic)

537.3171987533569


In [ ]:
counts = b.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x, y: x+y)

In [ ]:
counts.collect()

[(24, 84), (30, 46), (31, 3407), (19, 890), (23, 153), (29, 223)]

In [ ]:
from pyspark.sql import SparkSession
import random
spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

In [ ]:
player_raw = spark.read.csv("per_game_data/ablefo01_per_game.csv",sep=',',inferSchema=True, header=True)
player_raw.show()

+------+----+---+---+----+---+----+---+---+---+---+---+---+----+---+---+---+---+---------+
|Season| Age| Tm| Lg| Pos|  G|  GS| MP| FG|FGA|FG%| FT|FTA| FT%|TRB|AST| PF|PTS|PlayerKey|
+------+----+---+---+----+---+----+---+---+---+---+---+---+----+---+---+---+---+---------+
|  1957|24.0|SYR|NBA|null|1.0|null|1.0|0.0|2.0|0.0|0.0|0.0|null|1.0|1.0|1.0|0.0| ablefo01|
+------+----+---+---+----+---+----+---+---+---+---+---+---+----+---+---+---+---+---------+



In [ ]:
from pyspark.sql.functions import lit

def player_data_join(df1, df2):
  columns = list(set(df1.columns).union(set(df2.columns)))
  for column in set(columns) - set(df1.columns):
    df1 = df1.withColumn(column,lit(None))
  for column in set(columns) - set(df2.columns):
    df2 = df2.withColumn(column,lit(None))

  return df1.unionByName(df2)

In [ ]:
import os
per_game_data_files = os.listdir("per_game_data")

per_game_df = spark.read.csv("per_game_data/" + per_game_data_files[0],sep=',',inferSchema=True, header=True)
for file in per_game_data_files:
  new_df = spark.read.csv("per_game_data/" + file,sep=',',inferSchema=True, header=True)
  per_game_df = player_data_join(per_game_df, new_df)


In [ ]:
per_game_df.write.csv("per_game_full")